In [2]:
import fire_spread as fs
import pandas as pd
import datetime as dt

# DATE = 'Local Date'
# TIME = 'Local Time'
# TEMP = 'Temp (C)'
# RH = 'RH (%)'
# WIND_DIR = 'Wind Dir'
# WIND_SPEED = 'Wind Speed (km/h)'
# DF = 'Drought Factor'
# FFDI = 'FFDI'
# GFDI = 'GFDI'
# DATETIME = 'DateTime'

In [ ]:
# general model settings
weather_fn = 'data\TestPointForecast.csv'
start_date = '20210827'
start_time = '09:00'
duration = 24 #hours
slope = 0 #but note Cruz et al. for large fires slope effect negligible



In [ ]:

weather_data = fs.get_weather(weather_fn)
weather_data = fs.trim_weather(weather_data, start_date, start_time, duration)
weather_data

In [ ]:
weather_data.dtypes

In [ ]:
start_datetime = start_date + " " + start_time
start_datetime = dt.datetime.strptime(start_datetime,"%Y%m%d %H:%M")
print(start_datetime)